<a href="https://colab.research.google.com/github/mricos/bird2vec/blob/master/config/Wavegan_tf1_12p3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Training Wavegan with tensorflow 1.12.0 and python 3.6.7

Official implementation of WaveGAN, a machine learning algorithm which learns to generate raw audio waveforms.

https://github.com/chrisdonahue/wavegan/

@inproceedings {donahue2019wavegan,
  title={Adversarial Audio Synthesis},
  author={Donahue, Chris and McAuley, Julian and Puckette, Miller},
  booktitle={ICLR},
  year={2019}
}

### Mount Google Drive
  In this notebook we mount our google drive so that we can store the output of the model while it trains.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Install recommended dependencies
    pip install tensorflow-gpu==1.12.0
    pip install scipy==1.0.0
    pip install matplotlib==3.0.2
    pip install librosa==0.6.2
    
    It's worth it to note that this code maybe compatible different versions of TensorFlow, however we follow the recommendations of the author.

In [0]:
!pip install tensorflow-gpu==1.12.0
!pip install scipy==1.0.0
!pip install matplotlib==3.0.2
!pip install librosa==0.6.2

    100% |████████████████████████████████| 281.7MB 74kB/s 
    100% |████████████████████████████████| 3.1MB 7.6MB/s 
tensorflow 1.13.1 has requirement tensorboard<1.14.0,>=1.13.0, but you'll have tensorboard 1.12.2 which is incompatible.
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
    100% |████████████████████████████████| 50.0MB 988kB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
cvxpy 1.0.15 has requirement scipy>=1.1.0, but you'll have scipy 1.0.0 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: scipy 1.1.0
    Uninstalling scipy-1.1.0:
      Successfully uninstalled scipy-1.1.0


    100% |████████████████████████████████| 12.9MB 3.7MB/s 
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.50.post1 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: matplotlib 3.0.3
    Uninstalling matplotlib-3.0.3:
      Successfully uninstalled matplotlib-3.0.3


    100% |████████████████████████████████| 1.6MB 15.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/18/b8/10/f0f8f6ac60668a5cd75596cf14c25bb6b3ea1ecd815f058b7e
Successfully built librosa
  Found existing installation: librosa 0.6.3
    Uninstalling librosa-0.6.3:
      Successfully uninstalled librosa-0.6.3


## Install cuda9.0 for compatability

Here we download the CUDA toolkit from the Nvidia site, unpack it, add an apt-key to install the CUDA 9 libraires

In [0]:
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo dpkg -i /content/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!sudo apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
print('add key: ok')
!sudo apt-get update
!sudo apt-get install cuda-libraries-9-0

--2019-04-01 15:52:51--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?K5KLDRe9Zb3xgUwoq_lqFf05JlLU1AgtxgGXy8qORSdJzqhb4ck8V8F4q6Nry7UJFUf1RnFPQfj65fuLOBgJ7wtepejySaigXK9BmpgyC6-WLUNCTWEjlJ632-_zRBoNwxdUR_WTaEWM0MRHb-8Tou0NaU2ZbeBWa7Tl9i3DEOUNThPeLWLktwlOZVodWDC3JlYqSPhzgBSMIpkNqfvx [following]
--2019-04-01 15:52:52--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?K5KLDRe9Zb3xgUwoq_lqFf05JlLU1AgtxgGXy8qORSdJzqhb4ck8V8F4q6Nry7UJFUf1RnFPQfj65fuLOBgJ7wtepejySaigXK9BmpgyC6-WLU

## Clone wavegan, download unzip SC09 data

One of the main points of this notebook book is to train a model without having the data beforehand. In the next cell, we clone the repository that contains the model, download the data we want to train on and unzip the files in place to its own folder.

In [0]:
!git clone https://github.com/chrisdonahue/wavegan.git
!wget http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz #about 60 seconds
!tar -xzf /content/sc09.tar.gz #in /content

Cloning into 'wavegan'...
remote: Enumerating objects: 552, done.
remote: Total 552 (delta 0), reused 0 (delta 0), pack-reused 552
Receiving objects: 100% (552/552), 74.27 MiB | 40.71 MiB/s, done.
Resolving deltas: 100% (325/325), done.
--2019-04-02 17:07:48--  http://deepyeti.ucsd.edu/cdonahue/wavegan/data/sc09.tar.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 541227122 (516M) [application/octet-stream]
Saving to: ‘sc09.tar.gz’

sc09.tar.gz         100%[===================>] 516.15M  11.2MB/s    in 46s     

2019-04-02 17:08:34 (11.2 MB/s) - ‘sc09.tar.gz’ saved [541227122/541227122]



Before moving forward, we want to confirm that our cuda9 install is infact working. 
       - Note: At this point, the colab instance has both CUDA 9 and CUDA 10          installed. You can confirm this by opening the file explorer to the left. Following this path:
        ('>', Files> [^],>usr>local)
        The CUDA directories should be present. In any event, if CUDA 9 did not install properly, the next cell will throw an error.

In [0]:
# Confirm GPU is running
from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']
if len(get_available_gpus()) == 0:
    for i in range(4):
        print('WARNING: Not running on a GPU! See above for faster generation')
else:
    print('Training on GPU')

Training on GPU


### Start training

In the next cell the code to train the model will be executed. A few things to note:
        - The model is training directly from the '/content/' folder.
        - The output of the model will be stored in the afore linked google drive at ''/content/gdrive/My Drive/wgtrain.
        - The data is being read from '/content/SC09/train'
        
Since the data consists of small bits of audio, it is necessary to add additional flags:
        - 'data_first_slice' to signal small clips
        - 'data_fast_wav' to skip parsing with librosa and use scipy.
        
WARNING: If the data is a collection of MP3's do not use these flags

In [0]:
!export CUDA_VISIBLE_DEVICES="0"
!python /content/wavegan/train_wavegan.py train ./gdrive/My\ Drive/wgtran --data_dir /content/sc09/train/ --data_first_slice --data_fast_wav

Found 18620 audio files in specified directory
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d_transpose instead.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

--------------------------------------------------------------------------------
G

In [0]:
import tensorflow as tf
from IPython.display import display, Audio
import numpy as np

# Load the graph
tf.reset_default_graph()
saver = tf.train.import_meta_graph('/content/test/infer/infer.meta')

graph = tf.get_default_graph()
sess = tf.InteractiveSession()

saver.restore(sess, '/content/test/model.ckpt-947')

# Create 50 random latent vectors z
_z = (np.random.rand(50, 100) * 2.) - 1

# Synthesize G(z)
z = graph.get_tensor_by_name('z:0')
G_z = graph.get_tensor_by_name('G_z:0')
_G_z = sess.run(G_z, {z: _z})

# Play audio in notebook
display(Audio(_G_z[0, :, 0], rate=16000))

INFO:tensorflow:Restoring parameters from /content/test/model.ckpt-947


/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [0]:
!mkdir test
!cp -rf /content/gdrive/My\ Drive/wgtran/infer ./test/infer


cp: cannot stat '/content/gdrive/My Drive/wgtran/model.ckpt-947.{data-00000-of-00001,': No such file or directory
cp: cannot stat 'index,': No such file or directory
cp: cannot stat 'meta}': No such file or directory


In [0]:
!cp -rf /content/test /content/gdrive/My\ Drive/DS/bird2vec

In [0]:
# Generate and display audio

# CHANGE THESE to change number of examples generated/displayed
ngenerate = 64
ndisplay = 4

import numpy as np
import PIL.Image
from IPython.display import display, Audio
import time as time

# Sample latent vectors
_z = (np.random.rand(ngenerate, 100) * 2.) - 1.

# Generate
z = graph.get_tensor_by_name('z:0')
G_z = graph.get_tensor_by_name('G_z:0')[:, :, 0]
#G_z_spec = graph.get_tensor_by_name('G_z_spec:0')

start = time.time()
_G_z = sess.run([G_z], {z: _z})
print('Finished! (Took {} seconds)'.format(time.time() - start))

for i in range(ndisplay):
  print('-' * 80)
  print('Example {}'.format(i))
  #display(PIL.Image.fromarray(_G_z_spec[i]))
  display(Audio(_G_z[i], rate=16000))

Finished! (Took 0.7636475563049316 seconds)
--------------------------------------------------------------------------------
Example 0


--------------------------------------------------------------------------------
Example 1


IndexError: ignored